In [ ]:
import numpy as np
from tqdm import tqdm

In [2]:
data_path = 'CA Dataset'
data_checkin = data_path + "/checkin_CA_venues.txt"
data_friend = data_path + "/fs_friendship_CA.txt"

In [ ]:
def processing_time(string):
    week_day_dict = {'mon': 0, 'tue': 1, 'wed': 2, 'thu': 3, 'fri': 4, 'sat': 5, 'sun': 6}
    data_string = string.split()
    week_day = data_string[0].lower()
    week_day_num = week_day_dict[week_day]
    hour = int(data_string[3][:2]) + 1
    time_val = week_day_num * 24 + hour
    return time_val
    

In [4]:
def read_data_checkin(path):
    data_cate = None
    checkin_data = {}
    with open(path, 'r', encoding='utf-8') as file:
        count = 0
        for line in file:
            if count == 0:
                data_cate = line.split()
                for cate in data_cate:
                    checkin_data[cate] = []
                count += 1
                continue
            this_data = []
            data_line = line.strip().split('\t')
            for i, cate in enumerate(data_cate):
                checkin_data[cate].append(data_line[i])
            count += 1
    return checkin_data, data_cate

In [21]:
checkin_data, data_cate = read_data_checkin(data_checkin)
unique_venue = np.unique(checkin_data['VenueId'])
unique_venue_id2idx = {ele: i for i, ele in enumerate(unique_venue)}
unique_venue_cate = np.unique(checkin_data['VenueCategory'])
unique_venue_cate_id2idx = {ele: i for i, ele in enumerate(unique_venue_cate)}

In [6]:
unique_locations = np.unique(checkin_data['VenueLocation'])

In [27]:
location_checkin_dict = dict()
for i in tqdm(range(len(checkin_data['VenueId']))):
    venue = checkin_data['VenueId'][i]
    if venue not in location_checkin_dict:
        location_checkin_dict[venue] = 1
    else:
        location_checkin_dict[venue] += 1
location_checkin_dict = {k: v for k, v in sorted(location_checkin_dict.items(), key=lambda item: item[1]) if v > 7}
lol = np.array(list(location_checkin_dict.values()))
from collections import Counter
print(Counter(lol))
print("total number of checkins: {}".format(lol.sum()))
kept_venues = set(location_checkin_dict.keys())
print("number of kept venues: {}".format(len(kept_venues)))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 483813/483813 [00:00<00:00, 851417.37it/s]


Counter({8: 1595, 9: 1327, 10: 1054, 11: 921, 12: 735, 13: 645, 14: 482, 15: 465, 16: 413, 17: 348, 18: 344, 19: 330, 21: 252, 20: 246, 22: 214, 23: 211, 24: 176, 25: 167, 26: 149, 27: 144, 28: 131, 29: 125, 30: 104, 32: 102, 31: 101, 33: 83, 34: 81, 35: 72, 36: 68, 39: 64, 37: 60, 40: 59, 38: 58, 41: 58, 42: 58, 43: 47, 44: 43, 45: 39, 47: 36, 46: 32, 51: 30, 48: 29, 50: 29, 53: 28, 55: 28, 54: 26, 60: 26, 63: 26, 49: 24, 59: 23, 52: 22, 58: 21, 62: 21, 57: 20, 65: 18, 61: 16, 70: 15, 56: 14, 64: 14, 66: 14, 81: 14, 69: 13, 71: 13, 76: 13, 88: 12, 67: 11, 73: 11, 78: 11, 98: 11, 68: 10, 77: 10, 79: 10, 84: 10, 85: 10, 89: 10, 90: 10, 105: 10, 72: 9, 82: 9, 75: 8, 80: 8, 86: 8, 87: 7, 93: 7, 106: 7, 74: 6, 91: 6, 92: 6, 104: 6, 107: 6, 108: 6, 124: 6, 83: 5, 94: 5, 95: 5, 100: 5, 102: 5, 110: 5, 135: 5, 148: 5, 96: 4, 113: 4, 119: 4, 121: 4, 122: 4, 123: 4, 99: 3, 103: 3, 109: 3, 114: 3, 118: 3, 120: 3, 128: 3, 138: 3, 143: 3, 146: 3, 147: 3, 161: 3, 97: 2, 111: 2, 112: 2, 115: 2, 116:

In [30]:
selected_checkins = []
covered = set()
for i in tqdm(range(len(checkin_data['userID']))):
    # userID, time, venue, cate_venue
    venue_id = checkin_data['VenueId'][i]
    if venue_id not in kept_venues:
        continue
    user_id = int(checkin_data['userID'][i])
    try:
        user_id = renumbered_user[user_id]
    except:
        continue
    this_data = [user_id, processing_time(checkin_data['Time(GMT)'][i]), \
                 unique_venue_id2idx[checkin_data['VenueId'][i]], \
                 unique_venue_cate_id2idx[checkin_data['VenueCategory'][i]]]
    selected_checkins.append(this_data)
current_userIDs = [ele[0] for ele in selected_checkins]
kept_userIDs = [i for i in range(1, len(renumbered_user) + 1) if i not in current_userIDs]

selected_checkins = []
covered = set()
for i in tqdm(range(len(checkin_data['userID']))):
    # userID, time, venue, cate_venue
    venue_id = checkin_data['VenueId'][i]
    user_id = int(checkin_data['userID'][i])
    try:
        user_id = renumbered_user[user_id]
    except:
        continue
    if venue_id not in kept_venues and user_id not in kept_userIDs:
        continue
    this_data = [user_id, processing_time(checkin_data['Time(GMT)'][i]), \
                 unique_venue_id2idx[checkin_data['VenueId'][i]], \
                 unique_venue_cate_id2idx[checkin_data['VenueCategory'][i]]]
    selected_checkins.append(this_data)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 483813/483813 [00:03<00:00, 138326.50it/s]


In [32]:
selected_checkins[-1]

[3707, 28, 20932, 378]

In [34]:
current_userIDs = np.unique([ele[0] for ele in selected_checkins])

In [36]:
len(current_userIDs)

3707

In [63]:
kept_user = set()
for i in range(4164):
    if i not in selected_checkins[:, 0]:
        kept_user.add(i)

In [56]:
len(np.unique(checkin_data['userID']))

4163

In [95]:
len(np.unique(checkin_data['VenueCategory']))

636

In [45]:
len(checkin_data['VenueId'])

483813

In [71]:
len(np.unique(selected_checkins[:, 0]))

3842

In [75]:
np.save("selected_checkins.npy", selected_checkins)

In [37]:
friends = []
nodes = []
with open(data_friend, 'r', encoding='utf-8') as file:
    count = 0
    for line in file:
        if count == 0:
            count += 1
            continue
        data_line = line.strip().split(',')
        source = renumbered_user[int(data_line[0])]
        target = renumbered_user[int(data_line[1])]
        if source not in nodes:
            nodes.append(source)
        elif target not in nodes:
            nodes.append(target)
        if source < target:
            friends.append([source, target])
        


In [39]:
friends = np.array(friends)

In [41]:
friends.min()

1

In [43]:
to_reject_users = []
for i in tqdm(range(1, 3708)):
    not_appear = True
    for j in range(len(friends)):
        if i in friends[j]:
            not_appear = False
            break
    if not_appear:
        to_reject_users.append(i)
print(len(to_reject_users))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3707/3707 [01:53<00:00, 32.72it/s]

0


In [18]:
print(to_reject_users)

[7, 9, 22, 32, 41, 43, 45, 50, 52, 57, 73, 76, 77, 86, 96, 111, 112, 116, 129, 131, 135, 143, 150, 156, 158, 160, 170, 192, 207, 224, 238, 253, 254, 259, 270, 279, 302, 303, 307, 313, 326, 390, 448, 464, 486, 502, 517, 547, 559, 562, 564, 570, 589, 603, 623, 639, 671, 678, 699, 702, 720, 724, 734, 740, 776, 786, 796, 799, 800, 808, 814, 817, 822, 825, 839, 847, 875, 886, 893, 895, 897, 907, 911, 921, 929, 943, 952, 967, 968, 976, 1001, 1021, 1035, 1036, 1040, 1049, 1050, 1057, 1060, 1082, 1094, 1099, 1124, 1138, 1151, 1164, 1176, 1186, 1188, 1213, 1219, 1243, 1252, 1275, 1283, 1318, 1347, 1350, 1352, 1361, 1374, 1393, 1430, 1433, 1437, 1444, 1447, 1471, 1489, 1492, 1511, 1512, 1513, 1544, 1553, 1566, 1582, 1605, 1614, 1623, 1636, 1662, 1717, 1723, 1766, 1783, 1794, 1814, 1828, 1831, 1837, 1865, 1866, 1903, 1904, 1924, 1925, 1930, 1950, 1958, 1969, 1970, 1971, 1974, 2001, 2008, 2011, 2019, 2022, 2024, 2026, 2033, 2036, 2038, 2053, 2072, 2099, 2115, 2126, 2127, 2128, 2137, 2141, 2145, 21

In [19]:
renumbered_user = dict() 
count = 0
for i in range(1, 4164):
    if i in to_reject_users:
        count += 1
        continue
    renumbered_user[i] = i - count

In [20]:
renumbered_user

{1: 1,
 2: 2,
 3: 3,
 4: 4,
 5: 5,
 6: 6,
 8: 7,
 10: 8,
 11: 9,
 12: 10,
 13: 11,
 14: 12,
 15: 13,
 16: 14,
 17: 15,
 18: 16,
 19: 17,
 20: 18,
 21: 19,
 23: 20,
 24: 21,
 25: 22,
 26: 23,
 27: 24,
 28: 25,
 29: 26,
 30: 27,
 31: 28,
 33: 29,
 34: 30,
 35: 31,
 36: 32,
 37: 33,
 38: 34,
 39: 35,
 40: 36,
 42: 37,
 44: 38,
 46: 39,
 47: 40,
 48: 41,
 49: 42,
 51: 43,
 53: 44,
 54: 45,
 55: 46,
 56: 47,
 58: 48,
 59: 49,
 60: 50,
 61: 51,
 62: 52,
 63: 53,
 64: 54,
 65: 55,
 66: 56,
 67: 57,
 68: 58,
 69: 59,
 70: 60,
 71: 61,
 72: 62,
 74: 63,
 75: 64,
 78: 65,
 79: 66,
 80: 67,
 81: 68,
 82: 69,
 83: 70,
 84: 71,
 85: 72,
 87: 73,
 88: 74,
 89: 75,
 90: 76,
 91: 77,
 92: 78,
 93: 79,
 94: 80,
 95: 81,
 97: 82,
 98: 83,
 99: 84,
 100: 85,
 101: 86,
 102: 87,
 103: 88,
 104: 89,
 105: 90,
 106: 91,
 107: 92,
 108: 93,
 109: 94,
 110: 95,
 113: 96,
 114: 97,
 115: 98,
 117: 99,
 118: 100,
 119: 101,
 120: 102,
 121: 103,
 122: 104,
 123: 105,
 124: 106,
 125: 107,
 126: 108,
 127: 109,


In [75]:
friends.max()

3707

In [47]:
len(renumbered_user)

3707

In [48]:
index_cover = set()
seen = []
for i in range(1, len(renumbered_user) + 1):
    for j in range(len(friends)):
        if i in friends[j]:
            index_cover.add(j)
            break

In [73]:
index_cover

{0,
 8192,
 1,
 2,
 3,
 4,
 8198,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 8208,
 14,
 15,
 16,
 17,
 8213,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 8250,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 8259,
 8260,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 8283,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 8292,
 8293,
 93,
 8295,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 8303,
 103,
 104,
 8306,
 106,
 107,
 108,
 109,
 8311,
 8312,
 8313,
 8314,
 8315,
 8316,
 116,
 117,
 118,
 8320,
 120,
 121,
 122,
 123,
 124,
 125,
 8327,
 127,
 128,
 129,
 139,
 140,
 141,
 142,
 143,
 8336,
 136,
 137,
 147,
 148,
 144,
 150,
 145,
 152,
 146,
 154,
 155,
 8348,
 149,
 158,
 8351,
 159,
 160,
 162,
 163,
 156,
 157,
 166,
 167,
 168,
 169,
 8362,
 170,
 164,
 173,
 165,
 175,
 8367,
 171,
 172,
 179,
 1

In [51]:
len(friends)

16256

In [86]:
friend_test_percent = 0.55
num_test_friend = int(friend_test_percent * len(friends))
friendship_new_indices = np.random.choice([ele for ele in range(len(friends)) if ele not in index_cover], num_test_friend, replace=False)
friendship_old_indices = np.array([ele for ele in range(len(friends)) if ele not in friendship_new_indices])

In [87]:
for ele in list(index_cover):
    if ele not in friendship_old_indices:
        print(ele)

In [88]:
old_friendships = friends[friendship_old_indices]
new_friendships = friends[friendship_new_indices]

In [89]:
len(old_friendships)

7316

In [93]:
old_nodes = set([ele[0] for ele in old_friendships])

In [95]:
old_nodes.update(set([ele[1] for ele in old_friendships]))

In [98]:
for i in range(4000):
    if i not in old_nodes:
        print(i)

0
3708
3709
3710
3711
3712
3713
3714
3715
3716
3717
3718
3719
3720
3721
3722
3723
3724
3725
3726
3727
3728
3729
3730
3731
3732
3733
3734
3735
3736
3737
3738
3739
3740
3741
3742
3743
3744
3745
3746
3747
3748
3749
3750
3751
3752
3753
3754
3755
3756
3757
3758
3759
3760
3761
3762
3763
3764
3765
3766
3767
3768
3769
3770
3771
3772
3773
3774
3775
3776
3777
3778
3779
3780
3781
3782
3783
3784
3785
3786
3787
3788
3789
3790
3791
3792
3793
3794
3795
3796
3797
3798
3799
3800
3801
3802
3803
3804
3805
3806
3807
3808
3809
3810
3811
3812
3813
3814
3815
3816
3817
3818
3819
3820
3821
3822
3823
3824
3825
3826
3827
3828
3829
3830
3831
3832
3833
3834
3835
3836
3837
3838
3839
3840
3841
3842
3843
3844
3845
3846
3847
3848
3849
3850
3851
3852
3853
3854
3855
3856
3857
3858
3859
3860
3861
3862
3863
3864
3865
3866
3867
3868
3869
3870
3871
3872
3873
3874
3875
3876
3877
3878
3879
3880
3881
3882
3883
3884
3885
3886
3887
3888
3889
3890
3891
3892
3893
3894
3895
3896
3897
3898
3899
3900
3901
3902
3903
3904
3905
3906
390

In [90]:
len(new_friendships)

8940

In [86]:
len(np.unique(checkin_data['userID']))

4163

In [94]:
len(friends)

16256

In [100]:
old_friendships

array([[1721, 1791],
       [ 213, 2803],
       [2638, 3923],
       ...,
       [2044, 2439],
       [1319, 1655],
       [1289, 4105]])

In [44]:
np.save('CA Dataset/selected_checkins_new.npy', selected_checkins)

In [117]:
mydata = np.load('CA Dataset/selected_checkins_new.npy')

In [99]:
np.save('CA Dataset/old_friendship_new.npy', old_friendships)
np.save('CA Dataset/new_friendship_new.npy', new_friendships)

In [55]:
mydata = np.load('CA Dataset/selected_checkins_new.npy')
offset1 = max(mydata[:,0])
_, n = np.unique(mydata[:,1], return_inverse=True) # 
mydata[:,1] = n + offset1 + 1
offset2 = max(mydata[:,1])
_, n = np.unique(mydata[:,2], return_inverse=True)
mydata[:,2] = n + offset2 + 1
offset3 = max(mydata[:,2])
_, n = np.unique(mydata[:,3], return_inverse=True)
mydata[:,3] = n + offset3 + 1
n_nodes_total = np.max(mydata)

In [56]:
mydata

array([[    1,  3845, 12382, 17032],
       [    1,  3710,  7683, 17200],
       [    2,  3867, 15674, 17139],
       ...,
       [ 3707,  3829,  8969, 17180],
       [ 3707,  3758,  8969, 17180],
       [ 3707,  3735,  8969, 17180]])

In [57]:
from scipy.io import loadmat

In [10]:
examples = loadmat('dataset/dataset_connected_NYC.mat')['friendship_old']

In [11]:
examples

array([[   1,    2],
       [   1,   24],
       [   1,   30],
       ...,
       [3943, 3946],
       [3959, 3965],
       [3981, 3991]], dtype=uint16)

In [60]:
examples[:,3].min()

1

In [61]:
offset1 = max(examples[:,0])
_, n = np.unique(examples[:,1], return_inverse=True) # 
examples[:,1] = n + offset1 + 1
offset2 = max(examples[:,1])
_, n = np.unique(examples[:,2], return_inverse=True)
examples[:,2] = n + offset2 + 1
offset3 = max(examples[:,2])
_, n = np.unique(examples[:,3], return_inverse=True)
examples[:,3] = n + offset3 + 1
n_nodes_total = np.max(examples)

In [62]:
examples

array([[1801, 4063, 5176, 7882],
       [ 105, 4063, 4588, 7848],
       [ 571, 4063, 4536, 8055],
       ...,
       [ 718, 4083, 4915, 7882],
       [1446, 4083, 5959, 8041],
       [2305, 4084, 6713, 7980]])

In [63]:
examples[:, 0].max()

4024

In [64]:
mydata

array([[    1,  3845, 12382, 17032],
       [    1,  3710,  7683, 17200],
       [    2,  3867, 15674, 17139],
       ...,
       [ 3707,  3829,  8969, 17180],
       [ 3707,  3758,  8969, 17180],
       [ 3707,  3735,  8969, 17180]])

In [65]:
mydata[:, 3].max() - mydata[:, 3].min()

311

In [66]:
examples[:, 3].max() - examples[:, 3].min()

299

In [70]:
friends = loadmat('dataset/dataset_connected_NYC.mat')['friendship_old'].tolist()

In [74]:
for i in range(mydata[:, 0].max() + 1):
    flag = False
    for j in range(len(friends)):
        if i in friends[j]:
            flag = True
            break
    if flag == False:
        print(i)

0


In [100]:
oldfriend = np.load('CA Dataset/old_friendship_new.npy')

In [102]:
old_nodes = set(list(oldfriend[:, 0]) + list(oldfriend[:, 1]))

In [106]:
for i in range(4000):
    if i not in old_nodes:
        print(i)

0
3708
3709
3710
3711
3712
3713
3714
3715
3716
3717
3718
3719
3720
3721
3722
3723
3724
3725
3726
3727
3728
3729
3730
3731
3732
3733
3734
3735
3736
3737
3738
3739
3740
3741
3742
3743
3744
3745
3746
3747
3748
3749
3750
3751
3752
3753
3754
3755
3756
3757
3758
3759
3760
3761
3762
3763
3764
3765
3766
3767
3768
3769
3770
3771
3772
3773
3774
3775
3776
3777
3778
3779
3780
3781
3782
3783
3784
3785
3786
3787
3788
3789
3790
3791
3792
3793
3794
3795
3796
3797
3798
3799
3800
3801
3802
3803
3804
3805
3806
3807
3808
3809
3810
3811
3812
3813
3814
3815
3816
3817
3818
3819
3820
3821
3822
3823
3824
3825
3826
3827
3828
3829
3830
3831
3832
3833
3834
3835
3836
3837
3838
3839
3840
3841
3842
3843
3844
3845
3846
3847
3848
3849
3850
3851
3852
3853
3854
3855
3856
3857
3858
3859
3860
3861
3862
3863
3864
3865
3866
3867
3868
3869
3870
3871
3872
3873
3874
3875
3876
3877
3878
3879
3880
3881
3882
3883
3884
3885
3886
3887
3888
3889
3890
3891
3892
3893
3894
3895
3896
3897
3898
3899
3900
3901
3902
3903
3904
3905
3906
390

In [69]:
mydata[:, 0].max()

3707

In [71]:
oldfriend

array([[ 339,  697],
       [  86,  848],
       [1584, 2664],
       ...,
       [2239, 3133],
       [ 461, 1993],
       [1294, 1827]])

In [72]:
for i in range(1, 1000):
    not_in = True
    for j in range(len(oldfriend)):
        if i in oldfriend[j]:
            not_in = False
            break
    if not_in:
        print(i)

1
5
6
10
11
13
16
20
21
25
31
32
34
39
40
41
50
51
57
60
67
70
71
75
77
79
80
83
85
101
102
103
106
114
115
116
124
125
131
133
134
141
142
148
154
155
161
163
165
170
179
187
199
202
204
208
212
228
232
236
237
270
271
272
274
275
282
288
292
293
303
322
324
332
352
356
360
363
366
367
370
385
393
401
405
416
427
429
436
437
456
460
469
474
476
482
483
486
487
488
494
497
503
505
509
513
514
529
534
535
538
543
548
553
565
566
569
581
586
590
591
592
594
614
623
625
627
629
649
651
654
657
658
661
663
672
675
679
684
696
698
704
706
708
710
723
724
727
738
741
750
752
754
757
759
760
762
767
770
771
774
781
783
785
790
796
802
809
812
825
831
839
841
842
843
844
849
850
858
859
863
870
871
875
876
877
879
884
886
891
892
893
903
908
914
918
921
925
936
939
940
950
958
959
961
962
967
969
974
980
988
991


In [107]:
import pickle

In [108]:
file_renumbered_users = open('renumbered_user.pkl', 'wb')
pickle.dump(renumbered_user, file_renumbered_users)

In [111]:
selected_checkins

[[1, 138, 52246, 95],
 [1, 3, 13999, 436],
 [2, 160, 100571, 300],
 [2, 121, 27127, 14],
 [2, 95, 3423, 634],
 [2, 92, 23342, 634],
 [2, 164, 2702, 457],
 [2, 123, 11826, 66],
 [2, 160, 26502, 436],
 [2, 124, 2702, 457],
 [2, 44, 19161, 368],
 [2, 2, 2702, 457],
 [2, 138, 13434, 366],
 [2, 148, 2959, 295],
 [2, 69, 56192, 2],
 [2, 116, 13147, 66],
 [2, 100, 18863, 476],
 [2, 70, 63889, 222],
 [3, 21, 92411, 615],
 [4, 98, 4531, 37],
 [4, 145, 16501, 368],
 [4, 142, 28920, 436],
 [4, 147, 28920, 436],
 [4, 119, 9219, 66],
 [4, 147, 28920, 436],
 [4, 46, 2087, 480],
 [4, 46, 2087, 480],
 [4, 30, 2115, 480],
 [4, 29, 5938, 480],
 [4, 21, 59522, 13],
 [4, 67, 18137, 14],
 [4, 96, 43174, 8],
 [4, 118, 59522, 13],
 [4, 42, 72463, 436],
 [4, 27, 33270, 66],
 [4, 72, 9219, 66],
 [4, 142, 28004, 66],
 [4, 120, 28920, 436],
 [4, 93, 35532, 66],
 [4, 165, 28920, 436],
 [4, 91, 81005, 436],
 [4, 54, 28004, 66],
 [4, 121, 2671, 159],
 [4, 119, 6969, 213],
 [4, 92, 28920, 436],
 [4, 17, 20262, 312],

In [122]:
len(np.unique(mydata[:, 3]))

312

In [2]:
import numpy as np
from scipy.io import loadmat

In [3]:
hongzhi_checkins = np.load('CA Dataset/selected_checkins_new.npy')
hongzhi_old_friends = np.load('CA Dataset/old_friendship_new.npy')
hongzhi_new_friends = np.load('CA Dataset/new_friendship_new.npy')
hongzhi_friends = np.concatenate((hongzhi_old_friends, hongzhi_new_friends), axis=0)

In [4]:
hongzhi_friends

array([[   1,  212],
       [   1, 1155],
       [1155, 1966],
       ...,
       [2469, 3458],
       [ 766, 2453],
       [1487, 1558]])

In [5]:
hongzhi_checkins

array([[     1,    138,  52246,     95],
       [     1,      3,  13999,    436],
       [     2,    160, 100571,    300],
       ...,
       [  3707,    122,  20932,    378],
       [  3707,     51,  20932,    378],
       [  3707,     28,  20932,    378]])

In [7]:
with open('hongzhi_friends.csv', 'w', encoding='utf-8') as file:
    file.write('id1,id2\n')
    for i in range(len(hongzhi_old_friends)):
        friend_i = hongzhi_old_friends[i]
        file.write('{},{}\n'.format(friend_i[0], friend_i[1]))
        


In [15]:
mat_names = ['NYC', 'Jakarta', 'Istanbul', 'KualaLampur', 'Saopaulo', 'TKY']
for matn in mat_names:
    data_mat = loadmat('dataset/dataset_connected_{}.mat'.format(matn))['friendship_old']
    with open('{}_friends.csv'.format(matn), 'w', encoding='utf-8') as file:
        file.write('id1,id2\n')
        for i in range(len(data_mat)):
            friend_i = data_mat[i]
            file.write('{},{}\n'.format(friend_i[0], friend_i[1]))